In [1]:
import sys
sys.path.append(r'C:\Users\Administrator\PycharmProjects\backtester')

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

from backtester.core.context import Context
from datetime import timedelta
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from backtester.core.portfolio import OptionsPortfolio
from backtester.core.option_pricer import OPTION_TYPE_CALL, OPTION_TYPE_PUT
from backtester.strategies.straddle1 import StrategyStraddle1

def _groupbyInterpFunc(data):
    return interp1d(data['DaysToMaturity'], data['Volatility'], bounds_error=False, fill_value='extrapolate', kind='cubic')

def getMktData():
    filaname = r'C:\Users\Administrator\PycharmProjects\backtester\data\spx_vols.txt'

    df = pd.read_csv(filaname, parse_dates=['Date', 'Maturity']) # dtype={'Date': np.datetime64, 'Maturity': np.datetime64})
    df['Maturity'] = pd.to_datetime(df['Maturity'], format='%Y%m%d')
    df['DaysToMaturity'] = (df['Maturity'] - df['Date']).astype('timedelta64[D]') #TODO: check whether using day instead of business day to interpolate is correct?

    dfGroupedByInterp = pd.DataFrame(df.groupby(['Date', 'Spot']).apply(_groupbyInterpFunc))
    interpFuncDict = dfGroupedByInterp.to_dict('index')
    daysToMaturity = np.array(range(1,91))
    interpDict = {k: v[0](daysToMaturity) for k,v in interpFuncDict.items()}
    listOfDfs = [pd.DataFrame({'Date': k[0], 'Spot': k[1], 'DaysToMaturity':daysToMaturity, 'Volatility':v}) for k,v in interpDict.items()]
    dfInterpolatedVols = pd.concat(listOfDfs)
    dfInterpolatedVols['Maturity'] = dfInterpolatedVols['Date'] + pd.to_timedelta(dfInterpolatedVols['DaysToMaturity'],                                                                                  unit='d')
    dfInterpolatedVols['Stock'] = '.SPY'

    return dfInterpolatedVols




In [2]:
mktdata = getMktData()
context = Context(mktdata, balance=10000)

strategy = StrategyStraddle1(context)
strategy.run()

INFO:backtester.core.strategy:2013-02-05 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.strategy:2013-02-06 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.strategy:2013-02-07 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.strategy:2013-02-08 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.portfolio:2013-02-08 00:00:00: trading option: -29.6150928417884 of 2013-03-15 00:00:00 1517.93 call on .SPY @ 21.841628085697607
INFO:backtester.core.portfolio:2013-02-08 00:00:00: trading option: -29.6150928417884 of 2013-03-15 00:00:00 1517.93 put on .SPY @ 20.366579961389334
INFO:backtester.core.portfolio:2013-02-08 00:00:00: trading stock: -1.0705364378459823 on .SPY @ 1517.93
INFO:backtester.core.portfolio:2013-02-11 00:00:00: futures pnl adjustment: 
  Stock    Amount PreviousSpot     Spot       Date       PnL
0  .

INFO:backtester.core.strategy:2013-03-04 00:00:00: Balance: 16479.884111289262, Options: -3985.671532586576, Stock: 0, ref_notional: 12494.212578702685
INFO:backtester.core.portfolio:2013-03-04 00:00:00: trading stock: -18.028794220131438 on .SPY @ 1525.2
INFO:backtester.core.portfolio:2013-03-05 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -21.999566        1525.2  1539.79 2013-03-05 -320.973663
INFO:backtester.core.strategy:2013-03-05 00:00:00: Balance: 16158.910447911136, Options: -4320.054162098436, Stock: 0, ref_notional: 11838.8562858127
INFO:backtester.core.portfolio:2013-03-05 00:00:00: trading stock: -37.764645464270146 on .SPY @ 1539.79
INFO:backtester.core.portfolio:2013-03-06 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date        PnL
0  .SPY -59.764211       1539.79  1541.46 2013-03-06 -99.806233
INFO:backtester.core.strategy:2013-03-06 00:00:00: Balance: 16059.10421528796

INFO:backtester.core.portfolio:2013-03-22 00:00:00: trading stock: 10.8594747262256 on .SPY @ 1556.89
INFO:backtester.core.portfolio:2013-03-25 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date       PnL
0  .SPY -50.161962       1556.89  1551.69 2013-03-25  260.8422
INFO:backtester.core.strategy:2013-03-25 00:00:00: Balance: 16855.883986345278, Options: -5683.523128243243, Stock: 0, ref_notional: 11172.360858102034
INFO:backtester.core.portfolio:2013-03-25 00:00:00: trading stock: 4.400748441163046 on .SPY @ 1551.69
INFO:backtester.core.portfolio:2013-03-26 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -45.761213       1551.69  1563.77 2013-03-26 -552.795454
INFO:backtester.core.strategy:2013-03-26 00:00:00: Balance: 16303.088532574291, Options: -5922.4648563071905, Stock: 0, ref_notional: 10380.6236762671
INFO:backtester.core.portfolio:2013-03-26 00:00:00: trading stock: -29.26257328832

INFO:backtester.core.portfolio:2013-04-12 00:00:00: futures pnl adjustment: 
  Stock      Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -102.740966       1593.37  1588.85 2013-04-12  464.389168
INFO:backtester.core.strategy:2013-04-12 00:00:00: Balance: 15130.105902230502, Options: -3604.892159643362, Stock: 0, ref_notional: 11525.21374258714
INFO:backtester.core.portfolio:2013-04-12 00:00:00: trading option: -35.00209533667749 of 2013-05-17 00:00:00 1588.85 call on .SPY @ 21.35148910651185
INFO:backtester.core.portfolio:2013-04-12 00:00:00: trading option: -35.00209533667749 of 2013-05-17 00:00:00 1588.85 put on .SPY @ 19.807524488705212
INFO:backtester.core.portfolio:2013-04-12 00:00:00: trading stock: 34.58967522900788 on .SPY @ 1588.85
INFO:backtester.core.portfolio:2013-04-15 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date          PnL
0  .SPY -68.151291       1588.85  1552.36 2013-04-15  2486.840616
INFO:backtester.core.strat

INFO:backtester.core.strategy:2013-05-02 00:00:00: Balance: 24095.607794339998, Options: -6951.46964991494, Stock: 0, ref_notional: 17144.13814442506
INFO:backtester.core.portfolio:2013-05-02 00:00:00: trading stock: -40.23176618363801 on .SPY @ 1597.59
INFO:backtester.core.portfolio:2013-05-03 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date          PnL
0  .SPY -81.672411       1597.59  1614.42 2013-05-03 -1374.546679
INFO:backtester.core.strategy:2013-05-03 00:00:00: Balance: 22721.061115157958, Options: -8316.95933804442, Stock: 0, ref_notional: 14404.101777113538
INFO:backtester.core.portfolio:2013-05-03 00:00:00: trading option: -40.455902429651005 of 2013-06-07 00:00:00 1614.42 call on .SPY @ 23.037187726506318
INFO:backtester.core.portfolio:2013-05-03 00:00:00: trading option: -40.455902429651005 of 2013-06-07 00:00:00 1614.42 put on .SPY @ 21.468375467149144
INFO:backtester.core.portfolio:2013-05-03 00:00:00: trading stock: -38.3480059515

...

INFO:backtester.core.portfolio:2015-07-30 00:00:00: trading stock: -257.0051572051814 on .SPY @ 2108.63
INFO:backtester.core.portfolio:2015-07-31 00:00:00: expiring options
  Stock   Strike   Maturity  PutCall       Amount       Date     Spot  \
0  .SPY  2101.49 2015-07-31        1 -1031.019777 2015-07-31  2103.84   
1  .SPY  2101.49 2015-07-31       -1 -1031.019777 2015-07-31  2103.84   

      Moniness   Settlement  
0 -2422.896477 -2422.896477  
1  2422.896477     0.000000  
INFO:backtester.core.portfolio:2015-07-31 00:00:00: futures pnl adjustment: 
  Stock       Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -1450.360267       2108.63  2103.84 2015-07-31  6947.22568
INFO:backtester.core.strategy:2015-07-31 00:00:00: Balance: 1120238.4627870312, Options: -202025.62550035358, Stock: 0, ref_notional: 918212.8372866777
INFO:backtester.core.portfolio:2015-07-31 00:00:00: trading option: -2116.8319038954614 of 2015-09-04 00:00:00 2103.84 call on .SPY @ 28.13267041929248
IN

INFO:backtester.core.portfolio:2015-08-19 00:00:00: trading stock: 2074.3827458746837 on .SPY @ 2079.61
INFO:backtester.core.portfolio:2015-08-20 00:00:00: futures pnl adjustment: 
  Stock       Amount  PreviousSpot     Spot       Date            PnL
0  .SPY  2525.284566       2079.61  2035.73 2015-08-20 -110809.486745
INFO:backtester.core.strategy:2015-08-20 00:00:00: Balance: 1448492.9946162081, Options: -764492.9963613783, Stock: 0, ref_notional: 683999.9982548298
INFO:backtester.core.portfolio:2015-08-20 00:00:00: trading stock: 3051.1046288378348 on .SPY @ 2035.73
INFO:backtester.core.portfolio:2015-08-21 00:00:00: expiring options
  Stock   Strike   Maturity  PutCall       Amount       Date     Spot  \
0  .SPY  2126.64 2015-08-21        1 -1806.884405 2015-08-21  1970.89   
1  .SPY  2126.64 2015-08-21       -1 -1806.884405 2015-08-21  1970.89   

        Moniness     Settlement  
0  281422.246013       0.000000  
1 -281422.246013 -281422.246013  
INFO:backtester.core.portfolio:20

INFO:backtester.core.portfolio:2015-09-10 00:00:00: trading stock: 6.773322294663103 on .SPY @ 1952.29
INFO:backtester.core.portfolio:2015-09-11 00:00:00: expiring options
  Stock   Strike   Maturity  PutCall       Amount       Date     Spot  \
0  .SPY  2077.57 2015-09-11        1 -1810.092512 2015-09-11  1961.05   
1  .SPY  2077.57 2015-09-11       -1 -1810.092512 2015-09-11  1961.05   

        Moniness     Settlement  
0  210911.979512       0.000000  
1 -210911.979512 -210911.979512  
INFO:backtester.core.portfolio:2015-09-11 00:00:00: futures pnl adjustment: 
  Stock       Amount  PreviousSpot     Spot       Date           PnL
0  .SPY  4029.016507       1952.29  1961.05 2015-09-11  35294.184598
INFO:backtester.core.strategy:2015-09-11 00:00:00: Balance: 207182.75893380362, Options: -316950.91086835694, Stock: 0, ref_notional: -109768.15193455332
INFO:backtester.core.portfolio:2015-09-11 00:00:00: trading stock: -1816.3793959640852 on .SPY @ 1961.05
INFO:backtester.core.portfolio:2

INFO:backtester.core.portfolio:2015-10-02 00:00:00: expiring options
  Stock   Strike   Maturity  PutCall     Amount       Date     Spot  \
0  .SPY  1988.87 2015-10-02        1 -216.80657 2015-10-02  1951.36   
1  .SPY  1988.87 2015-10-02       -1 -216.80657 2015-10-02  1951.36   

      Moniness   Settlement  
0  8132.414442     0.000000  
1 -8132.414442 -8132.414442  
INFO:backtester.core.portfolio:2015-10-02 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date          PnL
0  .SPY  216.34951       1923.82  1951.36 2015-10-02  5958.265499
INFO:backtester.core.strategy:2015-10-02 00:00:00: Balance: -104500.37481490566, Options: 0.0, Stock: 0, ref_notional: -104500.37481490566


RuntimeError: found unexpected nulls 
   Stock  Amount  PreviousSpot   Spot   Date   PnL
0  False   False          True  False  False  True

In [15]:
df1 = pd.concat(strategy.optionsPortfolio.dfPositionsHist)
df2= pd.concat(strategy.stockPortfolio.dfPositionsHist)
df3 = pd.DataFrame(strategy.notionalDf, columns=['Date', 'Balance', 'OptionsNotional'])
df4 = pd.merge(df1, df2, how='outer', on=['Date'])
df5 = pd.merge(df4, df3, how='outer', on=['Date'])
#df5.info()
#df5
df5.to_csv('c:/temp/results2.csv')
#from IPython.display import display, HTML
#display(HTML(df.to_html()))

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
